# 5.0 简介

知识点：分类数据分为两类：
nominal（定类）：无内在顺序，如颜色、性别、品牌
ordinal（定序）：有内在顺序，如低/中/高、年轻/年迈
问题：机器学习算法需要数值型输入，无法直接处理字符串
示例：KNN算法计算距离时，字符串无法参与数学运算

# 5.1 对nominal型分类特征编码

对nominal分类特征（如州名）进行编码。

In [4]:
# 加载库
import numpy as np
from sklearn.preprocessing import LabelBinarizer

# 创建特征
feature = np.array([["Texas"],
                    ["California"],
                    ["Texas"],
                    ["Delaware"],
                    ["Texas"]])

# 创建one-hot编码器
one_hot = LabelBinarizer()

# 对特征进行one-hot编码
one_hot.fit_transform(feature)
# 输出:
# array([[0, 0, 1],
#        [1, 0, 0],
#        [0, 0, 1],
#        [0, 1, 0],
#        [0, 0, 1]])

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

查看分类类别

In [5]:
# 查看特征的分类
one_hot.classes_
# 输出: array(['California', 'Delaware', 'Texas'], dtype='<U10')

array(['California', 'Delaware', 'Texas'], dtype='<U10')

逆转换：

In [6]:
# 对one-hot编码逆转换
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

处理多标签分类：

In [7]:
# 创建有多个分类的特征
multiclass_feature = [("Texas", "Florida"),
                      ("California", "Alabama"),
                      ("Texas", "Florida"),
                      ("Delaware", "Florida"),
                      ("Texas", "Alabama")]

# 创建能处理多个分类的one-hot编码器
from sklearn.preprocessing import MultiLabelBinarizer
one_hot_multiclass = MultiLabelBinarizer()

# 对特征进行one-hot编码
one_hot_multiclass.fit_transform(multiclass_feature)
# 输出:
# array([[0, 0, 0, 1, 1],
#        [1, 1, 0, 0, 0],
#        [0, 0, 0, 1, 1],
#        [0, 0, 1, 1, 0],
#        [1, 0, 0, 0, 1]])

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

查看多标签分类：

In [8]:
# 查看分类
one_hot_multiclass.classes_
# 输出: array(['Alabama', 'California', 'Delaware', 'Florida', 'Texas'], dtype=object)

array(['Alabama', 'California', 'Delaware', 'Florida', 'Texas'],
      dtype=object)

讨论：
one-hot编码：将分类转换为二进制向量，避免序关系
虚拟变量陷阱：nominal特征编码后，为避免线性依赖，应删除一列
MultiLabelBinarizer：处理多标签分类（一个样本可属多个类别）

# 5.2 对ordinal分类特征编码

In [9]:
import warnings
# 忽略 scikit-learn 的弃用警告
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

In [10]:
# 加载库
import pandas as pd

# 创建特征
dataframe = pd.DataFrame({"Score": ["Low", "Low", "Medium", "Medium", "High"]})

# 创建映射器
scale_mapper = {"Low": 1,
                "Medium": 2,
                "High": 3}

# 使用映射器替换特征
dataframe["Score"].replace(scale_mapper)
# 输出:
# 0    1
# 1    1
# 2    2
# 3    2
# 4    3
# Name: Score, dtype: int64

C:\Users\24677\AppData\Local\Temp\ipykernel_15856\1350414191.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe["Score"].replace(scale_mapper)


0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

讨论：
映射方法：创建字典将字符串标签映射为数字
关键点：必须知道分类的顺序才能正确赋值
间隔不等的情况：当分类间隔不等时，应相应调整数值以反映实际差异

间隔不等的示例：

In [11]:
# 创建有细微差别的数据
dataframe = pd.DataFrame({"Score": ["Low", "Medium", "High", "Barely More Than Medium"]})

# 精确映射
scale_mapper = {"Low": 1,
                "Medium": 2,
                "Barely More Than Medium": 2.1,
                "High": 3}

dataframe["Score"].replace(scale_mapper)

C:\Users\24677\AppData\Local\Temp\ipykernel_15856\1938977125.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe["Score"].replace(scale_mapper)


0    1.0
1    2.0
2    3.0
3    2.1
Name: Score, dtype: float64

# 5.3对特征字典编码

问题描述：将字典转换为特征矩阵（例如词频向量）。

In [12]:
# 加载库
from sklearn.feature_extraction import DictVectorizer

# 创建字典
data = [{"Red": 4, "Blue": 3},
        {"Red": 3, "Blue": 4},
        {"Red": 1, "Yellow": 2},
        {"Red": 2, "Yellow": 2}]

# 创建字典向量化器
dictvectorizer = DictVectorizer()

# 将字典转换成特征矩阵
features = dictvectorizer.fit_transform(data)

# 查看特征矩阵
features.toarray()
# 输出:
# array([[4., 3., 0.],
#        [3., 4., 0.],
#        [1., 0., 2.],
#        [2., 0., 2.]])

array([[3., 4., 0.],
       [4., 3., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

获取特征名称

In [13]:
# 获取特征的名字
feature_names = dictvectorizer.get_feature_names_out()

# 查看特征的名字
feature_names
# 输出: array(['Blue', 'Red', 'Yellow'], dtype=object)

array(['Blue', 'Red', 'Yellow'], dtype=object)

转换为DataFrame：

In [14]:
# 加载库
import pandas as pd

# 从特征中创建DataFrame
pd.DataFrame(features.toarray(), columns=feature_names)
# 输出:
#    Blue  Red  Yellow
# 0   3.0  4.0     0.0
# 1   4.0  3.0     0.0
# 2   0.0  1.0     2.0
# 3   0.0  2.0     2.0

,Blue,Red,Yellow
0,3.0,4.0,0.0
1,4.0,3.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


讨论：
DictVectorizer：将字典转换为one-hot编码矩阵
应用场景：文本数据的词频统计、稀疏特征表示
sparse=False  ：强制输出稠密矩阵

# 5.4 填充缺失的分类值

问题描述：填充缺失的分类特征值。

In [15]:
# 加载库
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# 创建特征矩阵
X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])

# 创建带缺失值的特征矩阵
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                       [np.nan, -0.67, -0.22]])

# 训练KNN分类器（预测分类）
clf = KNeighborsClassifier(n_neighbors=3, weights='distance')
trained_model = clf.fit(X[:, 1:], X[:, 0])

# 预测缺失值的分类
imputed_values = trained_model.predict(X_with_nan[:, 1:])

# 将预测的分类和特征连接
X_with_imputed = np.hstack((imputed_values.reshape(-1, 1), X_with_nan[:, 1:]))

# 连接两个特征矩阵
np.vstack((X_with_imputed, X))
# 输出:
# array([[ 0.  ,  0.87,  1.31],
#        [ 1.  , -0.67, -0.22],
#        [ 0.  ,  2.1 ,  1.45],
#        [ 1.  ,  1.18,  1.33],
#        [ 0.  ,  1.22,  1.27],
#        [ 1.  , -0.21, -1.19]])

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

使用SimpleImputer（最频繁值填充）：

In [16]:
# 加载库
from sklearn.impute import SimpleImputer

# 连接两个特征矩阵
X_complete = np.vstack((X_with_nan, X))

# 创建填充器
imputer = SimpleImputer(strategy="most_frequent")

# 填充缺失值
imputer.fit_transform(X_complete)
# 输出:
# array([[ 0. ,  0.87,  1.31],
#        [ 0. , -0.67, -0.22],
#        [ 0. ,  2.1 ,  1.45],
#        [ 1. ,  1.18,  1.33],
#        [ 0. ,  1.22,  1.27],
#        [ 1. , -0.21, -1.19]])

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

讨论：
机器学习预测：将缺失分类特征作为目标变量，用其他特征预测
KNN算法：使用k个最近邻的距离加权预测
SimpleImputer：使用最频繁值填充，简单但可能引入偏差
推荐做法：预测后创建新特征，标识哪些观察值被填充

# 5.5 处理不均衡分类

问题描述：处理分类特征中类别不均衡的问题。

In [17]:
# 加载库
import numpy as np
from sklearn.datasets import load_iris

# 加载鸢尾花数据
iris = load_iris()

# 创建特征矩阵和目标向量
features = iris.data
target = iris.target

# 移除前40个观察值（使数据不均衡）
features = features[40:, :]
target = target[40:]

# 查看不均衡的目标向量
target
# 输出: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#              2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

策略1：使用分类权重：

In [18]:
# 创建带权重的随机森林分类器
from sklearn.ensemble import RandomForestClassifier

# 创建权重（类别0权重为0.9，类别1权重为0.1）
weights = {0: 0.9, 1: 0.1}

# 创建带权重的分类器
RandomForestClassifier(class_weight=weights)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [19]:
# 或使用自动均衡权重
RandomForestClassifier(class_weight='balanced')

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


策略2：下采样：

In [20]:
import numpy as np
from sklearn.datasets import load_iris

# 加载鸢尾花数据
iris = load_iris()
features = iris.data
target = iris.target

# 移除前40个观察值（使数据不均衡）
features = features[40:, :]
target = target[40:]

# 添加各类别索引
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]
i_class2 = np.where(target == 2)[0]

# 确定各类别样本数量
n_class0 = len(i_class0)  # 10
n_class1 = len(i_class1)  # 50
n_class2 = len(i_class2)  # 50

# 找到最小样本数量
min_class_size = min(n_class0, n_class1, n_class2)  # 10

# 从每个类别下采样到最小数量
i_class0_downsampled = np.random.choice(i_class0, size=min_class_size, replace=False)
i_class1_downsampled = np.random.choice(i_class1, size=min_class_size, replace=False)
i_class2_downsampled = np.random.choice(i_class2, size=min_class_size, replace=False)

# 合并索引并创建平衡数据集
balanced_indices = np.concatenate([i_class0_downsampled, i_class1_downsampled, i_class2_downsampled])
features_downsampled = features[balanced_indices]
target_downsampled = target[balanced_indices]

# 验证各类别数量
print("类别0数量:", len(np.where(target_downsampled == 0)[0]))
print("类别1数量:", len(np.where(target_downsampled == 1)[0]))
print("类别2数量:", len(np.where(target_downsampled == 2)[0]))

类别0数量: 10
类别1数量: 10
类别2数量: 10


策略3：上采样：

In [22]:
import numpy as np
from sklearn.datasets import load_iris

# 加载数据并制造不均衡
iris = load_iris()
features = iris.data
target = iris.target
features = features[40:, :]
target = target[40:]

# 获取各类别索引
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]
i_class2 = np.where(target == 2)[0]

# 确定样本数量
n_class0 = len(i_class0)
n_class1 = len(i_class1)

# 上采样：从少数类(class 0)中重复采样
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

# 创建平衡数据集（关键修正：features[i_class1] 而不是 features[i_class1:]）
features_upsampled = np.vstack([features[i_class0_upsampled],
                                features[i_class1],
                                features[i_class2]])

target_upsampled = np.concatenate([target[i_class0_upsampled],
                                   target[i_class1],
                                   target[i_class2]])

print("原始数据形状:", features.shape)
print("上采样后形状:", features_upsampled.shape)
print("各类别数量:", np.bincount(target_upsampled))

原始数据形状: (110, 4)
上采样后形状: (150, 4)
各类别数量: [50 50 50]


讨论：
不均衡问题：类别比例严重失衡时，准确率不是合适的评估指标
处理策略：
分类权重：通过class_weight参数调整各类别重要性
手动指定权重字典
balanced：自动设置与类别频数成反比的权重
下采样：从多数类中随机删除观察值，使各类别数量相等
优点：保留原始数据分布
缺点：可能丢失重要信息
上采样：从少数类中重复采样，使各类别数量相等
优点：保留所有原始数据
缺点：可能过拟合
评估标准：后续章节会讨论混淆矩阵、精确度、召回率、F1值、ROC曲线等更适合不均衡数据的指标
最佳实践：尝试多种方法，根据实际效果选择最佳方案。

# 知识点总结
分类数据类型：
nominal：无顺序（颜色、性别），需用one-hot编码
ordinal：有顺序（低/中/高），需保留序关系进行映射
编码方法：
LabelBinarizer：nominal特征one-hot编码
MultiLabelBinarizer：多标签分类编码
字典映射：ordinal特征编码，手动指定数值顺序
缺失值处理：
分类特征：使用KNN等机器学习算法预测
SimpleImputer：最频繁值填充（简单但可能偏差）
不均衡分类：
分类权重：class_weight参数调整
下采样：减少多数类样本
上采样：重复少数类样本
评估指标：避免使用准确率，应使用精确度、召回率、F1值等
实用工具：
DictVectorizer：将字典转换为特征矩阵（适用于文本词频）
KNN填充：利用其他特征预测缺失的分类值